<a href="https://colab.research.google.com/github/bo-cheng-tsai/week2/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# 定義股票代碼，這裡以台積電、鴻海和聯發科為例
stock_symbols = ["2330.TW", "2317.TW", "2454.TW"]

# 爬取所有股票的數據（今年的數據）
def download_stock_data(symbols):
    stocks_data = {}
    start_date = datetime(datetime.now().year, 1, 1).strftime('%Y-%m-%d')  # 今年1月1日
    end_date = datetime.now().strftime('%Y-%m-%d')  # 今天日期
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        # 獲取今年的數據
        data = stock.history(start=start_date, end=end_date)

        # 計算所需欄位
        data['成交股數'] = data['Volume']
        data['成交金額'] = data['Close'] * data['Volume']
        data['漲跌價差'] = data['Close'] - data['Open']
        data['成交筆數'] = data['Volume'] // 1000  # 假設每千股為一次交易

        # 選取所需欄位
        data = data[['成交股數', '成交金額', 'Open', 'High', 'Low', 'Close', '漲跌價差', '成交筆數']]
        data.columns = ['成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

        # 保存數據
        stocks_data[symbol] = data

    return stocks_data

# 將日期轉換為民國年並進行格式化
def convert_to_roc(date):
    year = date.year - 1911  # 將西元轉為民國年
    return f"{year}/{date.month:02}/{date.day:02}"

# 將數字格式化為有逗號分隔，並且保留兩位小數
def format_number(num, decimals=2):
    return f"{num:,.{decimals}f}"

# 爬取今年的股票數據
stocks_data = download_stock_data(stock_symbols)

# 將所有股票的數據合併為一個DataFrame
combined_data = pd.DataFrame()

for symbol, data in stocks_data.items():
    data['股票代碼'] = symbol  # 添加一列顯示股票代碼
    combined_data = pd.concat([combined_data, data])

# 重設索引
combined_data.reset_index(inplace=True)

# 轉換日期格式為民國年
combined_data['日期'] = combined_data['Date'].apply(convert_to_roc)

# 格式化數字，保留兩位小數且以逗號分隔
combined_data['成交股數'] = combined_data['成交股數'].apply(lambda x: format_number(x, 0))  # 成交股數為整數
combined_data['成交金額'] = combined_data['成交金額'].apply(lambda x: format_number(x, 2))
combined_data['開盤價'] = combined_data['開盤價'].apply(lambda x: format_number(x, 2))
combined_data['最高價'] = combined_data['最高價'].apply(lambda x: format_number(x, 2))
combined_data['最低價'] = combined_data['最低價'].apply(lambda x: format_number(x, 2))
combined_data['收盤價'] = combined_data['收盤價'].apply(lambda x: format_number(x, 2))
combined_data['漲跌價差'] = combined_data['漲跌價差'].apply(lambda x: format_number(x, 2))
combined_data['成交筆數'] = combined_data['成交筆數'].apply(lambda x: format_number(x, 0))  # 成交筆數為整數

# 選擇最終需要的欄位，並刪除 'Date' 欄位
combined_data = combined_data[['日期', '股票代碼', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']]

# 在Google Colab中顯示表格
display(combined_data.head(20))

# 保存為CSV文件
combined_data.to_csv('this_year_stock_prices.csv', index=False)

,日期,股票代碼,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/01/02,2330.TW,"26,059,058","15,253,956,847.97",582.40,585.36,581.41,585.36,2.96,"26,059"
1,113/01/03,2330.TW,"37,106,763","21,171,421,713.88",576.48,577.46,568.58,570.55,-5.92,"37,106"
2,113/01/04,2330.TW,"15,309,129","8,764,912,715.00",572.53,573.52,569.57,572.53,0.00,"15,309"
3,113/01/05,2330.TW,"18,158,971","10,324,828,041.51",570.55,572.53,566.61,568.58,-1.97,"18,158"
4,113/01/08,2330.TW,"17,761,275","10,221,432,724.11",574.50,577.46,571.54,575.49,0.99,"17,761"
5,113/01/09,2330.TW,"22,501,510","13,016,026,201.84",580.43,581.41,575.49,578.45,-1.97,"22,501"
6,113/01/10,2330.TW,"13,037,716","7,515,943,272.84",573.52,578.45,572.53,576.48,2.96,"13,037"
7,113/01/11,2330.TW,"23,997,409","13,881,330,822.70",578.45,581.41,575.49,578.45,0.00,"23,997"
8,113/01/12,2330.TW,"16,496,053","9,509,594,976.13",573.52,580.43,573.52,576.48,2.96,"16,496"
9,113/01/15,2330.TW,"21,143,076","12,230,238,379.71",582.40,582.40,577.46,578.45,-3.95,"21,143"


In [7]:
import requests
import pandas as pd
from tabulate import tabulate  # 確保你安裝了 tabulate

def get_stock_data(stock_no, date):
    """
    獲取指定股票在指定日期的歷史日線數據
    :param stock_no: 股票代號 (e.g., 2330 for 台積電)
    :param date: 查詢日期 (格式：YYYYMMDD，通常是查詢月份的第一天)
    :return: 該月份的歷史日線數據
    """
    url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock_no}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if data['stat'] == 'OK':
            df = pd.DataFrame(data['data'], columns=data['fields'])
            return df
        else:
            print("資料獲取失敗: ", data['stat'])
            return None
    else:
        print(f"HTTP 請求失敗，狀態碼: {response.status_code}")
        return None

# 設定要查詢的股票代號與日期 (日期格式：YYYYMMDD，通常填查詢月份的第一天)
stock_no = '2330'  # 例如台積電
date = '20240901'  # 查詢2024年8月的數據

# 獲取資料
stock_data = get_stock_data(stock_no, date)

# 檢查結果
if stock_data is not None:
    # 使用 tabulate 來打印數據，這樣可以形成更好看的表格
    print(tabulate(stock_data, headers='keys', tablefmt='pretty', showindex=False))
else:
    print("沒有獲取到數據")

+-----------+------------+----------------+----------+----------+--------+----------+----------+----------+
|   日期    |  成交股數  |    成交金額    |  開盤價  |  最高價  | 最低價 |  收盤價  | 漲跌價差 | 成交筆數 |
+-----------+------------+----------------+----------+----------+--------+----------+----------+----------+
| 113/09/02 | 19,272,593 | 18,270,058,260 |  950.00  |  955.00  | 943.00 |  948.00  |  +4.00   |  31,642  |
| 113/09/03 | 23,205,623 | 21,908,471,541 |  948.00  |  952.00  | 939.00 |  940.00  |  -8.00   |  47,139  |
| 113/09/04 | 93,169,835 | 83,424,133,824 |  894.00  |  905.00  | 888.00 |  889.00  |  -51.00  | 393,210  |
| 113/09/05 | 34,147,890 | 30,998,595,394 |  907.00  |  915.00  | 900.00 |  902.00  |  +13.00  |  63,902  |
| 113/09/06 | 28,248,063 | 25,786,016,936 |  909.00  |  918.00  | 903.00 |  918.00  |  +16.00  |  38,415  |
| 113/09/09 | 38,448,946 | 34,456,838,126 |  892.00  |  900.00  | 891.00 |  899.00  |  -19.00  | 102,365  |
| 113/09/10 | 34,312,646 | 31,067,424,234 |  907.00  |  91